In [1]:
from sqlalchemy import create_engine
import pymssql
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

model = LinearRegression()
qt = QuantileTransformer(output_distribution="normal")
poly = PolynomialFeatures(degree=2)
pipe = make_pipeline(poly, model)

from joblib import dump

In [12]:
def get_secrets(file_name):
    """
    helper function to read in secret login details 
    
    Inputs:
        file_name - a string pointing to the secret file path and file name

    Returns a list containing secret information
    """
    # open file and read contents line by line, saving to a list
    with open(file_name, mode='r') as file:
        secrets = []
        for line in file:
            secrets.append(line.strip())
    return secrets
    

In [14]:
def test_get_secrets():
    """ Implements unit tests for get_secrets() """
    
    def tests(file_name):
        # call get_secrets
        secrets = get_secrets(file_name)
        # returned type is a list
        is_list = type(secrets) == type([])
        # contents are strings
        contains_strings = False
        if len(secrets) != 0:
            contains_strings = True
        for secret in secrets:
            if type(secret) != type(str()):
                contains_strings = False
        # returned length
        list_len = len(secrets)
        # no leading or trailing whitespace
        whitespace = True
        for secret in secrets:
            if secret[0].isspace() or secret[-1].isspace():
                whitespace = False
        return is_list, contains_strings, list_len, whitespace
        
    # test 1
    file_name = 'test_get_secrets1.txt'
    test1 = tests(file_name)
    print(f"Test 1: input file: {file_name}")
    print(f"\tget_secrets() returned a list? {test1[0]}")
    print(f"\tget_secrets contains only strings? {test1[1]}")
    print(f"\tExpected length of 3. Pass? {3 == test1[2]}")
    print(f"\tLeading/Trailing whitespaces stripped? {test1[3]}")
    print(f"All pass? {test1[0] and test1[1] and test1[2] and test1[3]}")

# run tests
test_get_secrets()

Test 1: input file: test_get_secrets1.txt
	get_secrets() returned a list? True
	get_secrets contains only strings? True
	Expected length of 3. Pass? True
	Leading/Trailing whitespaces stripped? True
All pass? True


In [15]:
# get secrets
secrets = get_secrets('.secrets')
server = secrets[0]
username = secrets[1]
password = secrets[2]
dfbase = secrets[3]
string = "mssql+pymssql://" + username + ":" + password + "@" + server + "/" + dfbase
conn = create_engine(string).connect()

#### Get data

In [16]:
df = pd.read_sql(
    """
    select ticker, date, ret, bm, mom12m, roeq, mve
    from data
    where date>='2000-01'
    order by date, ticker
    """, 
    conn
)
df = df.dropna()
conn.close()

features = ["bm", "mom12m", "roeq"]
df = df.set_index(["date", "ticker"])

#### Drop largest 500 stocks each month

In [17]:
df["size_rnk"] = df.groupby("date").mve.rank(ascending=False)
df = df[df.size_rnk>500]

#### Transform features each month

In [18]:
def qt_df(d):
    x = qt.fit_transform(d)
    return pd.DataFrame(x, columns=d.columns, index=d.index)

df[features] = df.groupby("date", group_keys=False)[features].apply(qt_df)

#### Transform target each month

In [19]:
def qt_ser(s):
    x = s.copy()
    x = x.to_numpy().reshape(-1, 1)
    x = qt.fit_transform(x).flatten()
    return pd.Series(x, index=s.index)

df["target"] = df.groupby("date", group_keys=False).ret.apply(qt_ser)

#### Train and save

In [21]:
from datetime import date

In [22]:
today = date.today()

In [26]:
X = df[features]
y = df["target"]
pipe.fit(X, y)
model_file = 'files/linear_model_' + str(today) + '.joblib'
dump(pipe, model_file)

['files/linear_model_2023-01-31.joblib']